In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import json
import geopy.distance
import geopandas as gpd
from shapely.geometry import Point, Polygon


In [2]:
# Load the data
listings_df = pd.read_csv('data/listings.csv')
listingsgz_df = pd.read_csv('data/listings.csv.gz')
reviews_df = pd.read_csv('data/reviews.csv')
calendargz_df = pd.read_csv('data/calendar.csv.gz')
neighbourhoods_df = pd.read_csv('data/neighbourhoods.csv')
reviewsgz_df = pd.read_csv('data/reviews.csv.gz')
neighbourhoods_df_geojson = gpd.read_file('data/neighbourhoods.geojson')

#### Un grupo de inversionistas, solicitan un estudio sobre la ciudad de Munich cercano a Theresienwiese, donde se celebra todos los años el OctokerFest. Creen que invertir en esta zona, les dara mayores ganancias a futuro que en otras zonas de Munich. El objetivo de adquirir la propiedad, es convertirlo en un Airbnb, quieren determinar segun los resultados si será para compartir o departamento entero, según cual da mayores ganancias.
##### Se les brindará la informacion de los barrios más cercanos a este punto, con mayores precios de alquiler por Airbnb junto con el tipo de alquiler (compartido o entero).

In [3]:
# Obtener los valores únicos de la columna "id" en listings_df
listings_df_ids = listings_df['id'].unique()

# Obtener los valores únicos de la columna "id" en listingsgz_df
listingsgz_df_ids = listingsgz_df['id'].unique()

# Comparar los valores únicos de ambas columnas
if set(listings_df_ids) == set(listingsgz_df_ids):
    print("Las columnas 'id' en listings_df y listingsgz_df corresponden al mismo.")
else:
    print("Las columnas 'id' en listings_df y listingsgz_df no corresponden al mismo.")

Las columnas 'id' en listings_df y listingsgz_df corresponden al mismo.


In [4]:
sorted_listings_df = listingsgz_df.sort_values(by='price', ascending=False)
sorted_listings_df.head(1)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
1070,35144421,https://www.airbnb.com/rooms/35144421,20231226034302,2023-12-26,previous scrape,Rental unit in Munich · ★4.38 · Studio · 1 bed...,NaN,Inder Nachbarschaft befinden sich viele Restau...,https://a0.muscache.com/pictures/0874d86e-a713...,205832270,...,4.38,4.13,4.13,NaN,t,33,33,0,0,0.15


In [5]:
listings_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,97945,Rental unit in Munich · ★4.84 · 1 bedroom · 1 ...,517685,Angelika,NaN,Hadern,48.11492,11.48954,Entire home/apt,80.0,2,127,2019-10-03,0.85,2,72,0,NaN
1,114695,Rental unit in Munich · ★4.82 · 1 bedroom · 1 ...,581737,Stephan,NaN,Berg am Laim,48.12071,11.63758,Entire home/apt,105.0,2,66,2023-10-25,0.43,3,166,11,NaN
2,127383,Rental unit in Munich · ★4.83 · 1 bedroom · 1 ...,630556,Sonja,NaN,Maxvorstadt,48.15199,11.56482,Entire home/apt,120.0,3,110,2023-12-12,0.72,1,5,3,NaN
3,159634,Rental unit in Munich · ★4.63 · 1 bedroom · 1 ...,765694,Susana,NaN,Pasing-Obermenzing,48.13855,11.46586,Entire home/apt,60.0,3,37,2023-08-09,0.25,1,5,1,NaN
4,170154,Home in Munich · ★4.96 · 1 bedroom · 2 beds · ...,108297,Alex,NaN,Sendling-Westpark,48.10814,11.52733,Private room,70.0,3,546,2023-12-19,3.66,1,169,25,NaN


In [6]:
listings_df.isnull().sum()[listings_df.isnull().sum() > 0].sort_values(ascending=False)

neighbourhood_group    6555
license                6535
price                  1447
last_review            1314
reviews_per_month      1314
host_name                 1
dtype: int64

In [7]:
listingsgz_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,14726865,https://www.airbnb.com/rooms/14726865,20231226034302,2023-12-26,previous scrape,Rental unit in Munich · 1 bedroom · 1 bed · 1 ...,NaN,NaN,https://a0.muscache.com/pictures/7d11d1db-62ce...,91678611,...,NaN,NaN,NaN,NaN,f,1,0,0,1,NaN
1,14976237,https://www.airbnb.com/rooms/14976237,20231226034302,2023-12-26,previous scrape,Rental unit in Munich · 1 bedroom · 3 beds · 1...,NaN,NaN,https://a0.muscache.com/pictures/9f7cdeb8-3b33...,12448910,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
2,15794305,https://www.airbnb.com/rooms/15794305,20231226034302,2023-12-26,previous scrape,Rental unit in Munich · 1 bedroom · 1 bed · 1 ...,NaN,NaN,https://a0.muscache.com/pictures/c458bf85-45bc...,44987181,...,5.0,5.0,5.0,NaN,f,1,1,0,0,0.02
3,40839352,https://www.airbnb.com/rooms/40839352,20231226034302,2023-12-26,previous scrape,Rental unit in Munich · 1 bedroom · 1 bed · 1 ...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,317808258,...,5.0,5.0,5.0,NaN,f,1,1,0,0,0.05
4,45776129,https://www.airbnb.com/rooms/45776129,20231226034302,2023-12-26,previous scrape,Rental unit in Munich · 1 bedroom · 2 beds · 1...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,87534103,...,NaN,NaN,NaN,NaN,t,3,2,1,0,NaN


In [8]:
listingsgz_df.isnull().sum()[listingsgz_df.isnull().sum() > 0].sort_values(ascending=False)

description                     6555
bathrooms                       6555
calendar_updated                6555
bedrooms                        6555
neighbourhood_group_cleansed    6555
license                         6535
host_neighbourhood              4567
host_about                      3988
neighborhood_overview           3908
neighbourhood                   3908
host_response_rate              2310
host_response_time              2310
price                           1447
has_availability                1447
host_location                   1334
review_scores_checkin           1314
review_scores_accuracy          1314
review_scores_rating            1314
last_review                     1314
review_scores_communication     1314
review_scores_location          1314
review_scores_value             1314
first_review                    1314
review_scores_cleanliness       1314
reviews_per_month               1314
host_acceptance_rate            1177
beds                             107
h

In [9]:
listingsgz_df_lesscol = listingsgz_df.drop(['license', 'host_about', 'host_response_rate', 'host_response_time', "host_location","host_thumbnail_url","host_picture_url", "host_listings_count", "picture_url","description"], axis=1)

In [10]:
listingsgz_df_lesscol.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'neighborhood_overview', 'host_id', 'host_url', 'host_name',
       'host_since', 'host_acceptance_rate', 'host_is_superhost',
       'host_neighbourhood', 'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of

In [11]:
listingsgz_df_lesscol["price"].replace({'\$': '', ',': ''}, regex=True, inplace=True)
listingsgz_df_lesscol["price"] 

/var/folders/5n/nhlty2pd5q96xfw8z_1stsnm0000gn/T/ipykernel_43807/1495269943.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  listingsgz_df_lesscol["price"].replace({'\$': '', ',': ''}, regex=True, inplace=True)


0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
6550    120.00
6551     94.00
6552    135.00
6553    216.00
6554     90.00
Name: price, Length: 6555, dtype: object

In [12]:
listings_df.isnull().sum()[listings_df.isnull().sum() > 0].sort_values(ascending=False)

neighbourhood_group    6555
license                6535
price                  1447
last_review            1314
reviews_per_month      1314
host_name                 1
dtype: int64

In [13]:
listings_df_lesscol = listings_df.drop(["license", "neighbourhood_group", "host_name"], axis=1)

In [14]:
listings_df_lesscol.head()  

,id,name,host_id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,97945,Rental unit in Munich · ★4.84 · 1 bedroom · 1 ...,517685,Hadern,48.11492,11.48954,Entire home/apt,80.0,2,127,2019-10-03,0.85,2,72,0
1,114695,Rental unit in Munich · ★4.82 · 1 bedroom · 1 ...,581737,Berg am Laim,48.12071,11.63758,Entire home/apt,105.0,2,66,2023-10-25,0.43,3,166,11
2,127383,Rental unit in Munich · ★4.83 · 1 bedroom · 1 ...,630556,Maxvorstadt,48.15199,11.56482,Entire home/apt,120.0,3,110,2023-12-12,0.72,1,5,3
3,159634,Rental unit in Munich · ★4.63 · 1 bedroom · 1 ...,765694,Pasing-Obermenzing,48.13855,11.46586,Entire home/apt,60.0,3,37,2023-08-09,0.25,1,5,1
4,170154,Home in Munich · ★4.96 · 1 bedroom · 2 beds · ...,108297,Sendling-Westpark,48.10814,11.52733,Private room,70.0,3,546,2023-12-19,3.66,1,169,25


In [15]:
listings_df_lesscol["price"].isnull().sum()


np.int64(1447)

In [16]:
listings_df_lesscol["room_type"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 6555 entries, 0 to 6554
Series name: room_type
Non-Null Count  Dtype 
--------------  ----- 
6555 non-null   object
dtypes: object(1)
memory usage: 51.3+ KB


#### Valores nulos price en Listings

In [17]:
# Calculo mediana de precios en relacion a su room_type y neighbourhood
median_prices_roomnneigh = listings_df_lesscol.groupby(['room_type', 'neighbourhood'])['price'].median().reset_index()

median_prices_roomnneigh.rename(columns={'price': 'price_mediana'}, inplace=True)

listings_df_lesscol = listings_df_lesscol.merge(median_prices_roomnneigh, on=['room_type', 'neighbourhood'], how='left')

# Relleno de valores faltantes
listings_df_lesscol['price'].fillna(listings_df_lesscol['price_mediana'], inplace=True)

# Elimino la columna que hice para la mediana de precios
listings_df_lesscol.drop(columns=['price_mediana'], inplace=True)

#faltan 3 valores por rellenar pero no tiene tanta data como para completarlo, asi que lo dejo asi. 

/var/folders/5n/nhlty2pd5q96xfw8z_1stsnm0000gn/T/ipykernel_43807/3519924582.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  listings_df_lesscol['price'].fillna(listings_df_lesscol['price_mediana'], inplace=True)


In [18]:
listings_df_lesscol["price"].isnull().sum() 

np.int64(3)

#### Al df listings_df_lesscol voy a agregarle columnas de listingsgz_df_lesscol que sean utiles para el analisis

In [19]:
# para ver cual es el valor maximo y minimo del score de localizacion
max_value = listingsgz_df_lesscol["review_scores_location"].max()
min_value = listingsgz_df_lesscol["review_scores_location"].min()

print("Valor máximo:", max_value)
print("Valor mínimo:", min_value)

Valor máximo: 5.0
Valor mínimo: 1.0


In [20]:
# Agregar la columna "review_scores_location" a listings_df_lesscol
listings_df_lesscol = pd.merge(listings_df_lesscol, listingsgz_df_lesscol[["id", "review_scores_location","bedroom"]], on="id", how="left")

KeyError: "['bedroom'] not in index"

In [ ]:
listings_df_lesscol.head(1) # corroboro que se haya agregado la columna "review_scores_location"

,id,name,host_id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,review_scores_location
0,97945,Rental unit in Munich · ★4.84 · 1 bedroom · 1 ...,517685,Hadern,48.11492,11.48954,Entire home/apt,80.0,2,127,2019-10-03,0.85,2,72,0,4.78


#### Analisis de los barrios con precios mas altos

In [ ]:
max_prices_neigh =listings_df_lesscol .groupby(["id",'neighbourhood', 'room_type'])['price'].max().sort_values(ascending=False)
max_prices_neigh.head(10)

id                  neighbourhood                 room_type      
687443113487896053  Au-Haidhausen                 Shared room        96274.0
25961730            Feldmoching-Hasenbergl        Entire home/apt    11000.0
28192258            Ludwigsvorstadt-Isarvorstadt  Entire home/apt     8000.0
38989135            Sendling-Westpark             Entire home/apt     3582.0
662925540895249864  Maxvorstadt                   Entire home/apt     2000.0
974447132884521229  Neuhausen-Nymphenburg         Entire home/apt     1870.0
38261620            Altstadt-Lehel                Entire home/apt     1600.0
713516514999977755  Ludwigsvorstadt-Isarvorstadt  Entire home/apt     1500.0
817935207994522660  Neuhausen-Nymphenburg         Private room        1500.0
709166723659033290  Ludwigsvorstadt-Isarvorstadt  Entire home/apt     1500.0
Name: price, dtype: float64

In [ ]:
listings_df_lesscol[listings_df_lesscol["neighbourhood"] == "Au-Haidhausen"].sort_values("price", ascending=False)["price"]

3611    96274.0
1816      939.0
1677      700.0
1684      700.0
1683      700.0
         ...   
977        44.0
2226       43.0
2498       40.0
979        39.0
2998       35.0
Name: price, Length: 422, dtype: float64

#### Relacionar barrio con los precios mas altos y con los reviews scores mas altos


In [ ]:
listings_df_lesscol.columns

Index(['id', 'name', 'host_id', 'neighbourhood', 'latitude', 'longitude',
       'room_type', 'price', 'minimum_nights', 'number_of_reviews',
       'last_review', 'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'review_scores_location'],
      dtype='object')

In [ ]:
listings_df_lesscol["review_scores_location"].isnull().sum()

np.int64(1314)

In [ ]:
highest_review_scores =listings_df_lesscol["review_scores_location"].sort_values(ascending=False)
highest_review_scores

2927    5.0
3529    5.0
3501    5.0
3503    5.0
3504    5.0
       ... 
6550    NaN
6551    NaN
6552    NaN
6553    NaN
6554    NaN
Name: review_scores_location, Length: 6555, dtype: float64

In [ ]:
max_prices_neighreview =listings_df_lesscol .groupby(["id",'neighbourhood', 'room_type', "review_scores_location", 'latitude', 'longitude'])['price'].max().sort_values(ascending=False)
max_prices_neighreview.head(10)

id                  neighbourhood                 room_type        review_scores_location  latitude   longitude
25961730            Feldmoching-Hasenbergl        Entire home/apt  4.80                    48.210610  11.542590    11000.0
28192258            Ludwigsvorstadt-Isarvorstadt  Entire home/apt  5.00                    48.123280  11.566810     8000.0
38989135            Sendling-Westpark             Entire home/apt  4.00                    48.108340  11.526620     3582.0
662925540895249864  Maxvorstadt                   Entire home/apt  4.91                    48.151280  11.558864     2000.0
38261620            Altstadt-Lehel                Entire home/apt  5.00                    48.134160  11.576950     1600.0
12880683            Laim                          Entire home/apt  4.92                    48.137710  11.492780     1277.0
760646086436599544  Ludwigsvorstadt-Isarvorstadt  Entire home/apt  4.67                    48.126630  11.548525     1200.0
680922632192724403  Berg am

In [ ]:
dftop_10_max_prices_review = max_prices_neighreview.head(10)
dftop_10_max_prices_review


id                  neighbourhood                 room_type        review_scores_location  latitude   longitude
25961730            Feldmoching-Hasenbergl        Entire home/apt  4.80                    48.210610  11.542590    11000.0
28192258            Ludwigsvorstadt-Isarvorstadt  Entire home/apt  5.00                    48.123280  11.566810     8000.0
38989135            Sendling-Westpark             Entire home/apt  4.00                    48.108340  11.526620     3582.0
662925540895249864  Maxvorstadt                   Entire home/apt  4.91                    48.151280  11.558864     2000.0
38261620            Altstadt-Lehel                Entire home/apt  5.00                    48.134160  11.576950     1600.0
12880683            Laim                          Entire home/apt  4.92                    48.137710  11.492780     1277.0
760646086436599544  Ludwigsvorstadt-Isarvorstadt  Entire home/apt  4.67                    48.126630  11.548525     1200.0
680922632192724403  Berg am

In [ ]:
dftop_10_max_prices_review.describe()

count       10.00000
mean      3225.90000
std       3459.56235
min       1200.00000
25%       1200.00000
50%       1438.50000
75%       3186.50000
max      11000.00000
Name: price, dtype: float64

In [ ]:
dftop_10_max_prices_review.to_csv('top10munich.csv', index=False)
dftop_10_max_prices_review

id                  neighbourhood                 room_type        review_scores_location  latitude   longitude
25961730            Feldmoching-Hasenbergl        Entire home/apt  4.80                    48.210610  11.542590    11000.0
28192258            Ludwigsvorstadt-Isarvorstadt  Entire home/apt  5.00                    48.123280  11.566810     8000.0
38989135            Sendling-Westpark             Entire home/apt  4.00                    48.108340  11.526620     3582.0
662925540895249864  Maxvorstadt                   Entire home/apt  4.91                    48.151280  11.558864     2000.0
38261620            Altstadt-Lehel                Entire home/apt  5.00                    48.134160  11.576950     1600.0
12880683            Laim                          Entire home/apt  4.92                    48.137710  11.492780     1277.0
760646086436599544  Ludwigsvorstadt-Isarvorstadt  Entire home/apt  4.67                    48.126630  11.548525     1200.0
680922632192724403  Berg am